In [1]:
# Optuna tuning notebook (conservative setting)
# - Tuning XGBoost and TabNet with Optuna
# - Conservative search (A): n_trials = 30
# - Follow your pipeline: split 70/15/15, StandardScaler fit on train, SMOTE+Tomek on train only
# - Final evaluation uses validation to find threshold and test set for final metrics

# Note: This is a single-file script intended to be run as a Jupyter cell-by-cell notebook.
# If you want .ipynb, copy this code into a new notebook cell blocks.


In [2]:
!pip install numpy pandas torch optuna xgboost pytorch_tabnet scikit-learn imblearn

  Using cached numpy-2.3.5-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached optuna-4.6.0-py3-none-any.whl.metadata (17 kB)
  Using cached xgboost-3.1.2-py3-none-manylinux_2_28_x86_64.whl.metadata (2.1 kB)
  Using cached pytorch_tabnet-4.1.0-py3-none-any.whl.metadata (15 kB)
  Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached imblearn-0.0-py2.py3-none-any.whl.metadata (355 bytes)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.6.1-py3-none-any.whl.metadata (6.8 

In [3]:
import os
import time
import json
import numpy as np
import pandas as pd
import torch
import optuna
import xgboost as xgb
from xgboost import XGBClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report, roc_auc_score, f1_score, accuracy_score,
    precision_score, recall_score, confusion_matrix
)
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTETomek
from sklearn.utils import check_random_state
import warnings
warnings.filterwarnings('ignore')

/root/diabetes-prediction/venve/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# === USER EDITABLE CONFIG ===
DATA_PATH = "data/diabetes_binary.csv"  # <-- sesuaikan
SEED = 42
N_TRIALS = 30  # conservative (A)
N_FOLDS = 3
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # TabNet device; XGBoost stays CPU

# Reproducibility
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

print(f"PyTorch Version : {torch.__version__}")
print("Using Optuna for Bayesian-style search (TPE) | Conservative setting (n_trials=30)")

if torch.cuda.is_available():
    print(f"TabNet device set to GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA not available; TabNet will run on CPU (XGBoost stays on CPU).")


PyTorch Version : 2.9.1+cu128
Using Optuna for Bayesian-style search (TPE) | Conservative setting (n_trials=30)
TabNet device set to GPU: NVIDIA GeForce RTX 5090


In [5]:
# === LOAD DATA ===
print("Loading data...")
df = pd.read_csv(DATA_PATH)
X = df.iloc[:, 1:22].values
y = df.iloc[:, 0].values


Loading data...


In [6]:
# === SPLIT 70/15/15 ===
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=SEED
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=SEED
)

# === SCALING ===
scaler = StandardScaler()
X_train_scaled_before_balance = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# === BALANCE (SMOTE + TOMEK) on train only ===
print(f"Original Train Shape: {X_train.shape}")
print(f"Original Class dist : {np.bincount(y_train.astype(int))}")

smt = SMOTETomek(random_state=SEED, smote=SMOTE(random_state=SEED), tomek=TomekLinks())
X_train_bal, y_train_bal = smt.fit_resample(X_train_scaled_before_balance, y_train)

print(f"SMOTE+Tomek Train Shape   : {X_train_bal.shape}")
print(f"SMOTE+Tomek Class dist    : {np.bincount(y_train_bal.astype(int))}")

# Single stratified split of the balanced train for tuning (train/valid)
X_train_tune, X_val_tune, y_train_tune, y_val_tune = train_test_split(
    X_train_bal, y_train_bal, test_size=0.20, stratify=y_train_bal, random_state=SEED
)

# Utility: threshold search on validation
def find_best_threshold(y_val, y_prob_val):
    thresholds = np.arange(0.1, 0.9, 0.01)
    f1s = [f1_score(y_val, (y_prob_val > th).astype(int)) for th in thresholds]
    best_th = thresholds[np.argmax(f1s)]
    return best_th

# Metrics container
from collections import OrderedDict

def calculate_metrics(y_true, y_pred, y_prob):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    auc = roc_auc_score(y_true, y_prob)
    sensitivity = recall
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
    ppv = tp / (tp + fp) if (tp + fp) != 0 else 0
    npv = tn / (tn + fn) if (tn + fn) != 0 else 0
    return OrderedDict({
        'Accuracy': acc,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': auc,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv
    })


Original Train Shape: (177576, 21)
Original Class dist : [152834  24742]
SMOTE+Tomek Train Shape   : (304173, 21)
SMOTE+Tomek Class dist    : [152834 151339]


In [7]:
# === OPTUNA OBJECTIVE FOR XGBOOST ===

def objective_xgb(trial):
    # Suggest hyperparameters
    param = {
        'n_estimators': trial.suggest_categorical('n_estimators', [100, 200, 300]),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'objective': 'binary:logistic',
        'tree_method': 'hist',
        'random_state': SEED,
        'n_jobs': 1  # limit inside trials to avoid oversubscription
    }

    clf = XGBClassifier(**param)
    clf.fit(X_train_tune, y_train_tune, verbose=False)
    y_pred = clf.predict(X_val_tune)
    return float(f1_score(y_val_tune, y_pred))


In [8]:
# === OPTUNA OBJECTIVE FOR TABNET ===

def objective_tabnet(trial):
    # Suggest hyperparameters (conservative ranges)
    n_d = trial.suggest_categorical('n_d', [8, 16, 32])
    n_a = trial.suggest_categorical('n_a', [8, 16, 32])
    n_steps = trial.suggest_int('n_steps', 3, 8)
    gamma = trial.suggest_float('gamma', 1.0, 2.0)
    lambda_sparse = trial.suggest_loguniform('lambda_sparse', 1e-4, 1e-2)
    lr = trial.suggest_loguniform('lr', 1e-3, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [128, 256, 512])

    max_epochs = 50
    patience = 10

    clf = TabNetClassifier(
        n_d=n_d, n_a=n_a, n_steps=n_steps,
        gamma=gamma,
        lambda_sparse=lambda_sparse,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=lr),
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        scheduler_params={"step_size": 10, "gamma": 0.5},
        mask_type='entmax',
        device_name=DEVICE,
        seed=SEED,
        verbose=0
    )

    clf.fit(
        X_train=X_train_tune, y_train=y_train_tune,
        eval_set=[(X_val_tune, y_val_tune)],
        eval_name=['valid'],
        eval_metric=['auc'],
        max_epochs=max_epochs,
        patience=patience,
        batch_size=batch_size,
        virtual_batch_size=min(64, batch_size),
        num_workers=0,
        drop_last=False
    )

    y_prob = clf.predict_proba(X_val_tune)[:, 1]
    y_pred = (y_prob > 0.5).astype(int)
    score = float(f1_score(y_val_tune, y_pred))

    del clf
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

    return score


In [9]:
# === RUN STUDIES ===

# XGBoost study
print("Starting Optuna study for XGBoost...")
study_xgb = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=SEED))
study_xgb.optimize(objective_xgb, n_trials=N_TRIALS, n_jobs=1)

print("Best XGB trial:", study_xgb.best_trial.params)

# Save XGBoost study results
with open('optuna_xgb_best.json', 'w') as f:
    json.dump(study_xgb.best_trial.params, f, indent=2)


[I 2025-12-10 01:54:06,982] A new study created in memory with name: no-name-39e1abe6-a361-4b21-a9bf-b356f4701da8


Starting Optuna study for XGBoost...


[I 2025-12-10 01:54:11,120] Trial 0 finished with value: 0.885404741758333 and parameters: {'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.015958237752949748, 'subsample': 0.662397808134481, 'colsample_bytree': 0.6232334448672797, 'reg_alpha': 0.8661761457749352, 'reg_lambda': 0.6011150117432088}. Best is trial 0 with value: 0.885404741758333.
[I 2025-12-10 01:54:18,278] Trial 1 finished with value: 0.9063586984458799 and parameters: {'n_estimators': 300, 'max_depth': 9, 'learning_rate': 0.018891200276189388, 'subsample': 0.6727299868828402, 'colsample_bytree': 0.6733618039413735, 'reg_alpha': 0.3042422429595377, 'reg_lambda': 0.5247564316322378}. Best is trial 1 with value: 0.9063586984458799.
[I 2025-12-10 01:54:22,147] Trial 2 finished with value: 0.8877298473497999 and parameters: {'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.023993242906812727, 'subsample': 0.7465447373174767, 'colsample_bytree': 0.7824279936868144, 'reg_alpha': 0.7851759613930136, 'reg_lambda'

Best XGB trial: {'n_estimators': 200, 'max_depth': 9, 'learning_rate': 0.07898927623865823, 'subsample': 0.8854232900266661, 'colsample_bytree': 0.7067964291781478, 'reg_alpha': 0.0037214867687368793, 'reg_lambda': 0.7212484584766519}


In [10]:
# === TRAIN FINAL XGBOOST MODEL AND EVALUATE ===

best_xgb_params = study_xgb.best_trial.params
# map types: ensure required keys exist
xgb_final = XGBClassifier(
    objective='binary:logistic',
    tree_method='hist',
    n_jobs=-1,
    random_state=SEED,
    **best_xgb_params
)

start = time.time()
xgb_final.fit(X_train_bal, y_train_bal)
end = time.time()
print(f"XGBoost final trained in {(end-start)/60:.2f} mins")

# Find best threshold on validation
xgb_prob_val = xgb_final.predict_proba(X_val_scaled)[:, 1]
xgb_best_th = find_best_threshold(y_val, xgb_prob_val)

# Predict on test
y_prob_xgb = xgb_final.predict_proba(X_test_scaled)[:, 1]
y_pred_xgb = (y_prob_xgb > xgb_best_th).astype(int)

xgb_metrics = calculate_metrics(y_test, y_pred_xgb, y_prob_xgb)
print('\n--- XGBoost Metrics ---')
for k, v in xgb_metrics.items():
    print(f"{k:<12}: {v:.4f}")


XGBoost final trained in 0.03 mins

--- XGBoost Metrics ---
Accuracy    : 0.8173
Precision   : 0.3911
Recall      : 0.5588
F1-Score    : 0.4601
ROC-AUC     : 0.8265
Sensitivity : 0.5588
Specificity : 0.8591
PPV         : 0.3911
NPV         : 0.9232


In [11]:
# === RUN TABNET STUDY ===
print("Starting Optuna study for TabNet... (this may take a while)")
study_tab = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=SEED))
study_tab.optimize(objective_tabnet, n_trials=N_TRIALS, n_jobs=1)

print("Best TabNet trial:", study_tab.best_trial.params)

# Save TabNet study results
with open('optuna_tabnet_best.json', 'w') as f:
    json.dump(study_tab.best_trial.params, f, indent=2)


[I 2025-12-10 01:56:12,317] A new study created in memory with name: no-name-af2873b0-b396-44fb-a3fd-a3c076927db1


Starting Optuna study for TabNet... (this may take a while)

Early stopping occurred at epoch 16 with best_epoch = 6 and best_valid_auc = 0.84974


[I 2025-12-10 02:03:02,847] Trial 0 finished with value: 0.7816367265469062 and parameters: {'n_d': 16, 'n_a': 8, 'n_steps': 3, 'gamma': 1.866176145774935, 'lambda_sparse': 0.0015930522616241021, 'lr': 0.02607024758370768, 'batch_size': 256}. Best is trial 0 with value: 0.7816367265469062.



Early stopping occurred at epoch 46 with best_epoch = 36 and best_valid_auc = 0.89003


[I 2025-12-10 02:14:23,142] Trial 1 finished with value: 0.803976721629486 and parameters: {'n_d': 8, 'n_a': 16, 'n_steps': 4, 'gamma': 1.6118528947223796, 'lambda_sparse': 0.00019010245319870352, 'lr': 0.00383962929980417, 'batch_size': 512}. Best is trial 1 with value: 0.803976721629486.


...


Early stopping occurred at epoch 31 with best_epoch = 21 and best_valid_auc = 0.89096


[I 2025-12-10 07:51:14,598] Trial 28 finished with value: 0.8033690509097413 and parameters: {'n_d': 16, 'n_a': 32, 'n_steps': 7, 'gamma': 1.8597022329559751, 'lambda_sparse': 0.0003795559483484729, 'lr': 0.008893984862705692, 'batch_size': 512}. Best is trial 23 with value: 0.8095812452221184.



Early stopping occurred at epoch 29 with best_epoch = 19 and best_valid_auc = 0.8785


[I 2025-12-10 08:04:07,677] Trial 29 finished with value: 0.7962417389919579 and parameters: {'n_d': 16, 'n_a': 16, 'n_steps': 8, 'gamma': 1.7999132638568067, 'lambda_sparse': 0.0001197570628800693, 'lr': 0.026953525793715568, 'batch_size': 512}. Best is trial 23 with value: 0.8095812452221184.


Best TabNet trial: {'n_d': 16, 'n_a': 16, 'n_steps': 8, 'gamma': 1.6867971283087997, 'lambda_sparse': 0.000506447356892995, 'lr': 0.017194067952330448, 'batch_size': 512}


In [12]:
# === TRAIN FINAL TABNET MODEL AND EVALUATE ===

best_tab = study_tab.best_trial.params
clf_tabnet_final = TabNetClassifier(
    n_d=best_tab.get('n_d', 8),
    n_a=best_tab.get('n_a', 8),
    n_steps=best_tab.get('n_steps', 3),
    gamma=best_tab.get('gamma', 1.3),
    lambda_sparse=best_tab.get('lambda_sparse', 0.001),
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=best_tab.get('lr', 0.02)),
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    scheduler_params={"step_size": 10, "gamma": 0.5},
    mask_type='entmax',
    device_name=DEVICE,
    seed=SEED,
    verbose=1
)

start = time.time()
clf_tabnet_final.fit(
    X_train=X_train_bal, y_train=y_train_bal,
    eval_set=[(X_train_bal, y_train_bal), (X_val_scaled, y_val)],
    eval_name=['train', 'valid'],
    eval_metric=['auc', 'accuracy'],
    max_epochs=200,
    patience=20,
    batch_size=best_tab.get('batch_size', 256),
    virtual_batch_size=min(128, best_tab.get('batch_size', 256)),
    num_workers=0,
    drop_last=False
)
end = time.time()
print(f"TabNet final trained in {(end-start)/60:.2f} mins")

# Evaluate TabNet
tab_prob_val = clf_tabnet_final.predict_proba(X_val_scaled)[:, 1]
tab_best_th = find_best_threshold(y_val, tab_prob_val)

y_prob_tab = clf_tabnet_final.predict_proba(X_test_scaled)[:, 1]
y_pred_tab = (y_prob_tab > tab_best_th).astype(int)

tab_metrics = calculate_metrics(y_test, y_pred_tab, y_prob_tab)
print('\n--- TabNet Metrics ---')
for k, v in tab_metrics.items():
    print(f"{k:<12}: {v:.4f}")


epoch 0  | loss: 0.60215 | train_auc: 0.82054 | train_accuracy: 0.7452  | valid_auc: 0.79484 | valid_accuracy: 0.71079 |  0:00:32s
epoch 1  | loss: 0.50485 | train_auc: 0.83972 | train_accuracy: 0.75653 | valid_auc: 0.80927 | valid_accuracy: 0.75192 |  0:01:04s
epoch 2  | loss: 0.48637 | train_auc: 0.84742 | train_accuracy: 0.76737 | valid_auc: 0.81726 | valid_accuracy: 0.70367 |  0:01:37s
epoch 3  | loss: 0.4855  | train_auc: 0.84051 | train_accuracy: 0.75933 | valid_auc: 0.8055  | valid_accuracy: 0.74141 |  0:02:09s
epoch 4  | loss: 0.4837  | train_auc: 0.85602 | train_accuracy: 0.77453 | valid_auc: 0.8137  | valid_accuracy: 0.7284  |  0:02:42s
epoch 5  | loss: 0.4651  | train_auc: 0.86908 | train_accuracy: 0.78255 | valid_auc: 0.8164  | valid_accuracy: 0.73602 |  0:03:14s
epoch 6  | loss: 0.44169 | train_auc: 0.87589 | train_accuracy: 0.78744 | valid_auc: 0.81707 | valid_accuracy: 0.76075 |  0:03:45s
epoch 7  | loss: 0.43213 | train_auc: 0.87594 | train_accuracy: 0.78827 | valid_auc

In [13]:
# --- Summary DataFrame ---
metrics_df = pd.DataFrame({
    'Model': ['XGBoost', 'TabNet'],
    'Accuracy': [xgb_metrics['Accuracy'], tab_metrics['Accuracy']],
    'Precision': [xgb_metrics['Precision'], tab_metrics['Precision']],
    'Recall': [xgb_metrics['Recall'], tab_metrics['Recall']],
    'F1-Score': [xgb_metrics['F1-Score'], tab_metrics['F1-Score']],
    'ROC-AUC': [xgb_metrics['ROC-AUC'], tab_metrics['ROC-AUC']],
    'Sensitivity': [xgb_metrics['Sensitivity'], tab_metrics['Sensitivity']],
    'Specificity': [xgb_metrics['Specificity'], tab_metrics['Specificity']],
    'PPV': [xgb_metrics['PPV'], tab_metrics['PPV']],
    'NPV': [xgb_metrics['NPV'], tab_metrics['NPV']]
})

print('\n--- Summary ---')
print(metrics_df)

# Save metrics
metrics_df.to_csv('tuning_metrics_summary.csv', index=False)

print('\nSaved: optuna_xgb_best.json, optuna_tabnet_best.json, tuning_metrics_summary.csv')


--- Summary ---
     Model  Accuracy  Precision    Recall  F1-Score   ROC-AUC  Sensitivity  \
0  XGBoost  0.817276   0.391052  0.558846  0.460129  0.826541     0.558846   
1   TabNet  0.806896   0.376150  0.586005  0.458192  0.822811     0.586005   

   Specificity       PPV       NPV  
0     0.859115  0.391052  0.923249  
1     0.842656  0.376150  0.926323  

Saved: optuna_xgb_best.json, optuna_tabnet_best.json, tuning_metrics_summary.csv
